In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
df_foursq= pd.read_csv("datos/api_foursquare.csv", index_col=0)
df_aemet= pd.read_csv("datos/df_aemet_final.csv")
df_municipios= pd.read_csv("datos/municipios.csv", index_col=0)

In [3]:
df_municipios.sample(2)

,acebeda-la,ajalvir,alameda-del-valle,alamo-el,alcala-de-henares,alcobendas,alcorcon,aldea-del-fresno,algete,alpedrete,...,villamanta,villamantilla,villanueva-de-la-canada,villanueva-del-pardillo,villanueva-de-perales,villar-del-olmo,villarejo-de-salvanes,villaviciosa-de-odon,villavieja-del-lozoya,zarzalejo
2,La Acebeda,Ajalvir,Alameda del Valle,El Álamo,Alcalá de Henares,Alcobendas,Alcorcón,Aldea del Fresno,Algete,Alpedrete,...,Villamanta,Villamantilla,Villanueva de la Cañada,Villanueva del Pardillo,Villanueva de Perales,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo
1,-3.624399,-3.4807818,-3.8438216,-3.9905893,-3.3644973,-3.6358494,-3.8284295,-4.19936,-3.4974246,-4.0251223,...,-4.1090399,-4.1300303,-4.002795,-3.9600065,-4.1012616,-3.2355624,-3.2738764,-3.9002334,-3.6716291,-4.1815899


In [4]:
df_foursq["id_municipio_cambiar"]= df_foursq["municipio"]
df_foursq.drop(columns="municipio", inplace=True)

In [5]:
df_foursq.sample(2)

,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id_municipio_cambiar
469,TV Station,5583dc51498e37b88622ff8a,Unsure,551.0,/v3/places/5583dc51498e37b88622ff8a,Non Stop People,"Calle Gran Vía, 1, 28013 Madrid Comunidad de M...",40.419187,-3.69816,madrid
125,Park,4db4040c81543d71da3cc224,LikelyOpen,1460.0,/v3/places/4db4040c81543d71da3cc224,Parque de la República,Alcorcón Comunidad de Madrid,40.337376,-3.82117,alcorcon


In [6]:
df_aemet.sample(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
856,2024-08-30 06:00:00,Cubierto,15,15,13.0,0.0,0,100,Riesgo,0,20,Sin riesgo,N,10,berrueco-el-id28021
4876,2024-08-30 18:00:00,Intervalos nubosos con lluvia escasa,27,28,24.0,0.1,0,52,Riesgo,0,50,Sin riesgo,NE,15,pozuelo-de-alarcon-id28115


Separamos localización id 

In [7]:
df_aemet["localizacion_id"]

0       acebeda-la-id28001
1       acebeda-la-id28001
2       acebeda-la-id28001
3       acebeda-la-id28001
4       acebeda-la-id28001
               ...        
7880     zarzalejo-id28183
7881     zarzalejo-id28183
7882     zarzalejo-id28183
7883     zarzalejo-id28183
7884     zarzalejo-id28183
Name: localizacion_id, Length: 7885, dtype: object

In [8]:
df_aemet["id_municipio"]=pd.DataFrame(df_aemet["localizacion_id"].str.split("\.*-id\.*").str[1])
df_aemet["id_localizacion"]= pd.DataFrame(df_aemet["localizacion_id"].str.split("\.*-id\.*").str[0])
df_aemet.drop(columns="localizacion_id", inplace=True)



In [9]:
df_aemet.head(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,id_municipio,id_localizacion
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,28001,acebeda-la
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,28001,acebeda-la


Traspongo municipios para que se convierta el municipio en su id

In [10]:
df_municipioss= df_municipios.T
df_municipioss=df_municipioss.reset_index()

Nombro las columnas

In [11]:
lista= ["id_municipio_cambiar","latitude","longitude","nombre_municipio"]
df_municipioss.columns= lista
df_municipioss.head(2)

,id_municipio_cambiar,latitude,longitude,nombre_municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir


Meter id_municipio en tods las tablas

In [12]:
df_aemet.head(4)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,id_municipio,id_localizacion
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,28001,acebeda-la
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,28001,acebeda-la
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,28001,acebeda-la
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,28001,acebeda-la


In [13]:
df_municipioss.head(4)

,id_municipio_cambiar,latitude,longitude,nombre_municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo


Monto las tablas que no tienen foreign keys

In [ ]:
def crear_df_id (df, col):
    df_nuevo= pd.DataFrame(df[col])
    df_nuevo[f"id_{col}"]= pd.RangeIndex(start=1,stop = len(df_nuevo)+1,step=1)
    return df_nuevo

In [16]:
lista_cielo=[]
for nombre in df_aemet["cielo"].unique():
    lista_cielo.append(nombre)
lista_cielo   

In [33]:
df_cielo1= pd.DataFrame(lista_cielo)
df_cielo1= df_cielo1.rename(columns={0 : "cielo"})
df_cielo= crear_df_id(df_cielo1,"cielo")
df_cielo.head()

,cielo,id_cielo
0,Nubes altas,1
1,Muy nuboso,2
2,Cubierto con tormenta y lluvia escasa,3
3,Nuboso,4
4,Cielo despejado,5


In [34]:
lista_avisos=[]
for nombre in df_aemet["avisos"].unique():
    lista_avisos.append(nombre)
lista_avisos   

['Riesgo', 'Sin riesgo']

In [35]:
df_avisos1= pd.DataFrame(lista_avisos)
df_avisos1= df_avisos1.rename(columns={0 : "avisos"})
df_avisos= crear_df_id(df_avisos1,"avisos")
df_avisos.head()

,avisos,id_avisos
0,Riesgo,1
1,Sin riesgo,2


In [38]:
lista_categorias=[]
for nombre in df_foursq["category"].unique():
    lista_categorias.append(nombre)
lista_categorias

['Park',
 'Monument',
 'TV Station',
 'Castle',
 'Film Studio',
 'Palace',
 'Fountain',
 'Bridge',
 'Beach',
 'Print, TV, Radio and Outdoor Advertising Service']

In [39]:
df_categorias1= pd.DataFrame(lista_categorias)
df_categorias1= df_categorias1.rename(columns={0 : "categorias"})
df_categorias= crear_df_id(df_categorias1,"categorias")
df_categorias.head()

,categorias,id_categorias
0,Park,1
1,Monument,2
2,TV Station,3
3,Castle,4
4,Film Studio,5


In [40]:
lista_estados=[]
for nombre in df_foursq["closed_bucket"].unique():
    lista_estados.append(nombre)
lista_estados

['LikelyOpen', 'VeryLikelyOpen', 'Unsure']

In [41]:
df_estados1= pd.DataFrame(lista_estados)
df_estados1= df_estados1.rename(columns={0 : "estados"})
df_estados= crear_df_id(df_estados1,"estados")
df_estados.head()

,estados,id_estados
0,LikelyOpen,1
1,VeryLikelyOpen,2
2,Unsure,3


Pasamos a las siguientes tablas con menos ids

In [42]:
df= pd.DataFrame()
# df["id_cielo"]= df_cielo["id_cielo"]
# df["id_avisos"]= df_avisos["id_avisos"]
df["fecha"]= df_aemet["fecha"]
df["temperatura_°c"]= df_aemet["temp._(°c)"]
df["precipitacion_mm"]= df_aemet["precipitación_(mm)"]
df["humedad_%"]= df_aemet["humedad_relativa_(%)"]
df["avisos"]= df_aemet["avisos"]
df["cielo"]= df_aemet["cielo"]
# df["id_tiempo"]= pd.RangeIndex(start=1,stop = len(df)+1,step=1)
df_tiempo= df
df_tiempo

,fecha,temperatura_°c,precipitacion_mm,humedad_%,avisos,cielo
0,2024-08-29 12:00:00,21,0.0,69,Riesgo,Nubes altas
1,2024-08-29 13:00:00,24,0.0,57,Riesgo,Nubes altas
2,2024-08-29 14:00:00,23,0.0,59,Riesgo,Muy nuboso
3,2024-08-29 15:00:00,23,0.0,55,Riesgo,Nubes altas
4,2024-08-29 16:00:00,21,0.0,65,Riesgo,Nubes altas
...,...,...,...,...,...,...
7880,2024-08-31 03:00:00,17,0.0,90,Sin riesgo,Poco nuboso
7881,2024-08-31 04:00:00,17,0.0,91,Sin riesgo,Cielo despejado
7882,2024-08-31 05:00:00,17,0.0,88,Sin riesgo,Cielo despejado
7883,2024-08-31 06:00:00,16,0.0,89,Sin riesgo,Poco nuboso


Comprobamos nulos y tipos de cada DF

In [14]:
df_aemet["fecha"]= pd.to_datetime(df_aemet["fecha"])

In [40]:
df_aemet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7885 entries, 0 to 7884
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index                  7885 non-null   int64         
 1   fecha                  7885 non-null   datetime64[ns]
 2   cielo                  7885 non-null   object        
 3   temp._(°c)             7885 non-null   int64         
 4   sen._térmica_(°c)      7885 non-null   int64         
 5   racha_máx._(km/h)      7885 non-null   float64       
 6   precipitación_(mm)     7885 non-null   float64       
 7   nieve_(mm)             7885 non-null   int64         
 8   humedad_relativa_(%)   7885 non-null   int64         
 9   prob._precip.__(%)     7885 non-null   object        
 10  prob._de_nieve_(%)     7885 non-null   int64         
 11  prob._de_tormenta_(%)  7885 non-null   int64         
 12  avisos                 7885 non-null   object        
 13  dir

In [42]:
df_foursq["address"].fillna("sin_informacion", inplace= True)

In [43]:
df_foursq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 0 to 906
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   category       907 non-null    object 
 1   fsq_id         907 non-null    object 
 2   closed_bucket  907 non-null    object 
 3   distance       907 non-null    float64
 4   link           907 non-null    object 
 5   name           907 non-null    object 
 6   address        907 non-null    object 
 7   latitude       907 non-null    float64
 8   longitude      907 non-null    float64
 9   id_municipio   907 non-null    object 
dtypes: float64(3), object(7)
memory usage: 77.9+ KB


Traspongo municipios para que se convierta el municipio en su id

Transformo las coordenadas a float

In [33]:
df_municipioss["latitude"]= df_municipioss["latitude"].astype(float,)
df_municipioss["longitude"]= df_municipioss["longitude"].astype(float)

In [34]:
df_municipioss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_municipio      176 non-null    object 
 1   latitude          176 non-null    float64
 2   longitude         176 non-null    float64
 3   nombre_municipio  176 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.6+ KB


**NUEVO COMIENZO TENIENDO CLARA LA ESTRUCTURA**

EMPEZAMS POR MUNICIPIO

In [45]:
df_municipioss

,id_municipio,latitude,longitude,nombre_municipio
0,acebeda-la,41.086958,-3.624399,La Acebeda
1,ajalvir,40.534230,-3.480782,Ajalvir
2,alameda-del-valle,40.917718,-3.843822,Alameda del Valle
3,alamo-el,40.230730,-3.990589,El Álamo
4,alcala-de-henares,40.481840,-3.364497,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.336447,-3.235562,Villar del Olmo
172,villarejo-de-salvanes,40.168358,-3.273876,Villarejo de Salvanés
173,villaviciosa-de-odon,40.357379,-3.900233,Villaviciosa de Odón
174,villavieja-del-lozoya,41.006362,-3.671629,Villavieja del Lozoya
